# Required Libraries

In [1]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 95.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 67.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 97.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.0/120.0 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.6/201.6 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/

In [ ]:
import os
import json
import torch
import chromadb
import requests
import numpy as np
from PIL import Image
from io import BytesIO
from chromadb.config import Settings
from transformers import AutoProcessor, Gemma3ForConditionalGeneration
from transformers import CLIPVisionModel, RobertaModel, AutoTokenizer, CLIPFeatureExtractor

2025-08-20 06:34:38.902994: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755671679.250736      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755671679.350938      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

# Download Data

In [3]:
!git clone https://github.com/FaSha20/Natural-Language-Processing-Projects

Cloning into 'Natural-Language-Processing-Projects'...
remote: Enumerating objects: 95, done.
remote: Counting objects: 100% (95/95), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 95 (delta 30), reused 49 (delta 11), pack-reused 0 (from 0)
Receiving objects: 100% (95/95), 8.12 MiB | 16.91 MiB/s, done.
Resolving deltas: 100% (30/30), done.


In [4]:
with open("/kaggle/working/Natural-Language-Processing-Projects/MultimodalRAG/scientists_data/scintists_data_with_text_chunks.json", "r", encoding="utf-8") as f:
    data = json.load(f)

print(len(data), "records loaded")

300 records loaded


# Build Embeddings and DATABASE

## Load Pre-trained Encoder Model

In [5]:
vision_encoder = CLIPVisionModel.from_pretrained('SajjadAyoubi/clip-fa-vision')
preprocessor = CLIPFeatureExtractor.from_pretrained('SajjadAyoubi/clip-fa-vision')
text_encoder = RobertaModel.from_pretrained('SajjadAyoubi/clip-fa-text')
tokenizer = AutoTokenizer.from_pretrained('SajjadAyoubi/clip-fa-text')

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/350M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/clip/feature_extraction_clip.py:30: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(


config.json:   0%|          | 0.00/728 [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/473M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

## Encoding Functions

In [6]:
# ---- Text Encoder ----
def embed_text(text: str):
    text_embedding = text_encoder(**tokenizer(text, return_tensors='pt')).pooler_output
    return text_embedding[0]

# ---- Image Encoder ----
def embed_image(url: str):
    try:
        headers = {"User-Agent": "ImageDatasetScript/1.0 (email@gmail.com)"}
        response = requests.get(url, headers=headers, timeout=10)
        image = Image.open(BytesIO(response.content)).convert("RGB")
        image_embedding = vision_encoder(**preprocessor(image, return_tensors='pt')).pooler_output
        return image_embedding[0]
    except Exception as e:
        return None

## Build the DATABASE

In [7]:
PERSIST_DIR = "/content/chroma_scientists_db"
os.makedirs(PERSIST_DIR, exist_ok=True)

client = chromadb.PersistentClient(path=PERSIST_DIR, settings=Settings(allow_reset=True))

text_db  = client.get_or_create_collection(name="scientists_text",  metadata={"hnsw:space": "cosine"})
image_db = client.get_or_create_collection(name="scientists_image", metadata={"hnsw:space": "cosine"})

In [8]:
def upsert_pair(idx, name, text, image_url, t_emb, i_emb):
    meta = {
        "pair_id": idx,
        "name": name,
        "text": text,
        "image_url": image_url if image_url is not None else "",
    }

    if t_emb is not None:
        text_db.add(
            ids=[f"text-{idx}"],
            embeddings=[t_emb.tolist()],
            metadatas=[meta],
        )

    if i_emb is not None:
        image_db.add(
            ids=[f"img-{idx}"],
            embeddings=[i_emb.tolist()],
            metadatas=[meta],
        )

In [9]:
for idx, person in enumerate(data):
    name = person.get("name")

    text = person.get("text_chunk", "").replace("\u200c", "")
    t_emb = embed_text(text)

    image_urls = list(person.get("image", {}).values())
    img_url = None
    for url in image_urls:
        if url:
            i_emb = embed_image(url)
            if i_emb is not None:
              img_url = url
              break

    upsert_pair(idx, name, text, img_url, t_emb, i_emb)

## Retrieval Functions

In [10]:
def search_by_text(query: str, k=3):
    q_emb = embed_text(query)
    res = text_db.query(query_embeddings=[q_emb.tolist()], n_results=k, include=["metadatas", "distances"])
    return res

def search_by_image(image_url: str, k=5):
    q_emb = embed_image(image_url)
    if q_emb is None:
        return None
    res = image_db.query(query_embeddings=[q_emb.tolist()], n_results=k, include=["metadatas", "distances"])
    return res

## Test an example

In [11]:
res = search_by_text("چه کسی شرح کتاب مابعدالطبیعه ارسطو را نوشت؟", k=7)
for meta, dist in zip(res["metadatas"][0], res["distances"][0]):
    print("name:", meta["name"], "score:", 1-dist)
    print("text:", meta["text"])
    print("image:", meta["image_url"], "\n")

name: بنیامین بن موسی نهاوندی score: 0.8839471936225891
text: بنیامین بن موسی نهاوندی، حکیم، فیلسوف، عالم دینی، پزشک و عالم یهودی برجسته، در حدود قرن دوم شمسی در نهاوند واقع در استان همدان چشم به جهان گشود. او که در دوران قرون وسطی می زیست، نقشی اساسی در تکمیل و توسعه مبانی فرقه قرائیم ایفا نمود، جنبشی که توسط عنان ابن داوود پایه گذاری شده بود.  از جمله آثار ارزشمند او می توان به شروحی بر عهد عتیق و نوشته هایی به زبان عبری اشاره کرد.  نهاوندی با تاکید بر اهمیت تفکر آزاد و تحقیق، استدلال می کرد که تحقیق یک وظیفه است و اشتباهات ناشی از آن گناه محسوب نمی شود.
image:  

name: محمد بن طاهر بن بهرام سجستانی سیستانی score: 0.8695303201675415
text: محمد بن طاهر بن بهرام سجستانی سیستانی، معروف به "منطقی"، در حدود سال ۲۹۱ هجری شمسی در سیستان چشم به جهان گشود. این فیلسوف و اندیشمند برجسته سده چهارم هجری، تحصیلات خود را در زمینه فلسفه و منطق در بغداد و نزد اساتیدی چون ابوبشر متی بن یونس و ابوزکریا یحی بن عدی گذراند. سجستانی، علاوه بر فعالیتهای علمی، از حمایت سیاسی حاکمان محلی و دیلمی نیز برخوردار 

In [12]:
query_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/e/e6/Birjandimanuscript.jpg/220px-Birjandimanuscript.jpg"
res = search_by_image(query_url, k=5)
for meta, dist in zip(res["metadatas"][0], res["distances"][0]):
    print("name:", meta["name"], "score:", 1-dist)
    print("text:", meta["text"])
    print("image:", meta["image_url"], "\n")

name: نظام الدین عبدالعلی بیرجندی score: 1.0000001192092896
text: نظامالدین عبدالعلی بیرجندی، معروف به فاضل بیرجندی و محقق بیرجندی، دانشمند برجسته عصر صفوی بود. وی که در بیرجند واقع در خراسان جنوبی دیده به جهان گشود، در زمینههای ریاضیات و ستارهشناسی تبحر داشت. بیرجندی آثار متعددی از جمله "اسطرلاب"، "مختصر الهیئه" و "شرح التذکرة النصیریة فی الهیئة" را به رشته تحریر درآورد. او نزد اساتید بزرگی همچون غیاثالدین جمشید کاشانی و کمالالدین قنوی به تحصیل علوم مختلف پرداخت و سرانجام در سال ۹۳۴ هجری شمسی در بیرجند دار فانی را وداع گفت و در بجد به خاک سپرده شد.
image: https://upload.wikimedia.org/wikipedia/commons/thumb/e/e6/Birjandimanuscript.jpg/220px-Birjandimanuscript.jpg 

name: علی بن عباس مجوسی اهوازی score: 0.8311041593551636
text: علی بن عباس مجوسی اهوازی، پزشک و روانشناس برجسته سده چهارم هجری (دهم میلادی)، در شهر اهواز در ایران متولد شد. او که با القابی همچون "ابن المجوس" و "مجوسی اهوازی" نیز شناخته میشد، اثر ارزشمندی به نام "کتاب ملکی" از خود به یادگار گذاشته است. وی در حدود سال ۳۸۳ هجر

# Generator

In [ ]:
os.environ["TORCH_COMPILE_DISABLE"] = "1"
model_id = "google/gemma-3-4b-it"

model = Gemma3ForConditionalGeneration.from_pretrained(model_id, device_map="auto").eval()
processor = AutoProcessor.from_pretrained(model_id)

In [16]:
def load_image_from_url(url, timeout=10):
    headers = {"User-Agent": "MM-RAG/1.0 (email@example.com)"}
    resp = requests.get(url, headers=headers, timeout=timeout)
    resp.raise_for_status()
    return Image.open(BytesIO(resp.content)).convert("RGB")

def trim(s, n=300):
    s = " ".join(str(s).split())
    return (s[: n-1] + "…") if len(s) > n else s

def build_context_block(text_hits, image_hits, max_items=4):
    """
    text_hits / image_hits: lists of dicts with keys: name, text, image_url, distance
    Returns a compact Persian context string.
    """
    bullets = []
    # Prefer text facts first, then image captions/metadata (if any)
    for h in text_hits[:max_items]:
        bullets.append(f"- {h.get('name','?')}: {trim(h.get('text',''))} [src: متن] (d={h['distance']:.3f})")
    for h in image_hits[:max_items]:
        # we only add image entries that also have some text/metadata to avoid empty lines
        if h.get("text"):
            bullets.append(f"- {h.get('name','?')}: {trim(h.get('text',''))} [src: تصویر] (d={h['distance']:.3f})")
    header = "دانش بازیابی‌شده (خلاصه):"
    return header + "\n" + "\n".join(bullets) if bullets else ""


In [17]:
def _postprocess_query_result(res):
    out = []
    if not res:
        return out
    md = res.get("metadatas", [[]])[0]
    ds = res.get("distances", [[]])[0]
    for m, d in zip(md, ds):
        m = dict(m) if m else {}
        m["distance"] = float(d)
        out.append(m)
    return out

def retrieve_for_text_query(query: str, k_text=5, k_image=5):
    q_emb = embed_text(query)
    t_res = text_db.query(query_embeddings=[q_emb.tolist()], n_results=k_text, include=["metadatas", "distances"])
    i_res = image_db.query(query_embeddings=[q_emb.tolist()], n_results=k_image, include=["metadatas", "distances"])
    text_hits  = _postprocess_query_result(t_res)
    image_hits = _postprocess_query_result(i_res)
    return text_hits, image_hits

def retrieve_for_image_query(image_url: str, k_text=5, k_image=5):
    q_emb = embed_image(image_url)
    if q_emb is None:
        return [], []
    t_res = text_db.query(query_embeddings=[q_emb.tolist()], n_results=k_text, include=["metadatas", "distances"])
    i_res = image_db.query(query_embeddings=[q_emb.tolist()], n_results=k_image, include=["metadatas", "distances"])
    text_hits  = _postprocess_query_result(t_res)
    image_hits = _postprocess_query_result(i_res)
    return text_hits, image_hits

In [18]:
def gemma_rag_generate(
    question_text: str,
    text_hits,
    image_hits,
    force_image: bool = True,
    max_new_tokens: int = 256,
    temperature: float = 0.9,
    top_p: float = 0.9,
):
    # Pick best retrieved image
    best_img = None
    best_img_url = None
    if image_hits:
        best = sorted(image_hits, key=lambda x: x["distance"])[0]
        if best.get("image_url"):
            try:
                best_img = load_image_from_url(best["image_url"])
                best_img_url = best["image_url"]
            except Exception:
                best_img = None

    # Build retrieval context
    context_block = build_context_block(text_hits, image_hits, max_items=4)

    # Persian system prompt
    system_prompt = (
        "تو یک دستیار چندوجهی هستی. با تکیه بر «دانش بازیابی‌شده» و در صورت وجود «تصویر»، "
        "به پرسش زیر پاسخ بده. اگر پاسخ در داده‌های بازیابی‌شده نبود، بگو «اطلاعات کافی ندارم»."
        " پاسخ را به فارسی، دقیق و کوتاه بده."
    )

    # Build user content
    user_content = []
    if best_img is not None and force_image:
        user_content.append({"type": "image", "image": best_img})
    if context_block:
        user_content.append({"type": "text", "text": context_block})
    user_content.append({"type": "text", "text": f"پرسش: {question_text}"})

    # Messages for Gemma
    messages = [
        {"role": "system", "content": [{"type": "text", "text": system_prompt}]},
        {"role": "user", "content": user_content},
    ]

    # Apply chat template
    inputs = processor.apply_chat_template(messages, add_generation_prompt=True,
                                           tokenize=True,return_dict=True,
                                           return_tensors="pt").to(model.device, dtype=torch.bfloat16)

    input_len = inputs["input_ids"].shape[-1]

    with torch.inference_mode():
        out = model.generate(**inputs, max_new_tokens=max_new_tokens,
                             do_sample=True, temperature=temperature, top_p=top_p)[0][input_len:]

    text = processor.decode(out, skip_special_tokens=True)
    return {
        "answer": text.strip(),
        "used_image_url": best_img_url,
        "context": context_block,
        "question": question_text
    }

In [20]:
def rag_qa_from_text(query: str, k_text=5, k_image=5, force_image=True):
    t_hits, i_hits = retrieve_for_text_query(query, k_text=k_text, k_image=k_image)
    return gemma_rag_generate(query, t_hits, i_hits, force_image=force_image)

def rag_qa_from_image(image_url: str, question_text: str, k_text=5, k_image=5, force_image=True):
    t_hits, i_hits = retrieve_for_image_query(image_url, k_text=k_text, k_image=k_image)
    return gemma_rag_generate(question_text, t_hits, i_hits, force_image=force_image)

In [21]:
question = "سلام! میشه یک توضیح کوتاه درباره‌ی مریم میرزاخانی بدی؟"
result = rag_qa_from_text(question, k_text=5, k_image=5, force_image=False)

print("📌 پرسش:", result["question"])
print("📖 زمینه:", result["context"])
print("🤖 پاسخ:", result["answer"])

skipping cudagraphs due to skipping cudagraphs due to multiple devices: device(type='cuda', index=0), device(type='cuda', index=1)


📌 پرسش: سلام! میشه یک توضیح کوتاه درباره‌ی مریم میرزاخانی بدی؟
📖 زمینه: دانش بازیابی‌شده (خلاصه):
- علیرضا یلدا: علیرضا یلدا، پژوهشگر، نویسنده و استاد زبان و ادبیات فارسی معاصر، در سال ۱۳۲۶ در فردوس متولد شد. متاسفانه اطلاعات بیشتری در مورد زندگی و آثار ایشان در دست نیست، اما نام ایشان در میان فعالان عرصه فرهنگ و ادب ایران میدرخشد. [src: متن] (d=0.452)
- ابوبکر عبدالله بن احمد بن محمّد اسفرزی: ابوبکر عبدالله بن احمد بن محمّد اسفرزی، پزشکی برجسته در دوره تیموریان، در شهر اسفراین واقع در خراسان چشم به جهان گشود. از جمله آثار ارزشمند به جا مانده از این شخصیت میتوان به کتاب "زبده البیان فی علم الابدان" اشاره کرد که نشاندهنده تبحر و دانش او در زمینه پزشکی است. اطلاعات دقیقی در مورد تاریخ تولد… [src: متن] (d=0.464)
- توفیق موسیوند: توفیق موسیوند، پزشک برجسته و مخترع قلب مصنوعی، در ۲۹ خرداد سال ۱۳۱۶ در شهر همدان متولد شد. او که متخصص جراحی قلب و عروق، مهندس پزشکی و استاد جراحی بود، به دلیل اختراع اولین قلب مصنوعی جهان شهرت یافت. موسیوند پس از ازدواج و فرزنددار شدن، در سن ۳۷ سالگی به تحصیل در 